In [ ]:
#dbfs:/FileStore/shared_uploads/danghia2002@gmail.com/u.data
#dbfs:/FileStore/shared_uploads/danghia2002@gmail.com/u.item
#dbfs:/FileStore/shared_uploads/danghia2002@gmail.com/u.genre
#dbfs:/FileStore/shared_uploads/danghia2002@gmail.com/u_occupation
#dbfs:/FileStore/shared_uploads/danghia2002@gmail.com/u.user

In [ ]:
# Nạp dữ liệu từ file u.item vào DataFrame
item_data = spark.read.format("csv").option("delimiter", "|").load("dbfs:/FileStore/shared_uploads/danghia2002@gmail.com/u.item")
item_data = item_data.selectExpr("_c0 as item_id", "_c1 as title", "_c2 as release_date", "_c4 as imdb_url")
item_data = item_data.withColumn("item_id", item_data["item_id"].cast("integer"))
# Nạp dữ liệu từ file u.data vào DataFrame
rating_data = spark.read.format("csv").option("delimiter", "\t").load("dbfs:/FileStore/shared_uploads/danghia2002@gmail.com/u.data")
rating_data = rating_data.selectExpr("_c0 as user_id", "_c1 as item_id", "_c2 as rating", "_c3 as timestamp")
rating_data = rating_data.withColumn("user_id", rating_data["user_id"].cast("integer"))
rating_data = rating_data.withColumn("item_id", rating_data["item_id"].cast("integer"))
rating_data = rating_data.withColumn("rating", rating_data["rating"].cast("integer"))
# Nạp dữ liệu từ file u.user vào DataFrame
user_data = spark.read.format("csv").option("delimiter", "|").load("dbfs:/FileStore/shared_uploads/danghia2002@gmail.com/u.user")
user_data = user_data.selectExpr("_c0 as user_id", "_c1 as age", "_c2 as gender", "_c3 as occupation", "_c4 as zip_code")
user_data = user_data.withColumn("user_id", user_data["user_id"].cast("integer"))
# Nạp dữ liệu từ file u.occupation vào DataFrame
occupation_data = spark.read.format("csv").option("delimiter", "|").load("dbfs:/FileStore/shared_uploads/danghia2002@gmail.com/u_occupation")
occupation_data = occupation_data.selectExpr("_c0 as occupation")
# Nạp dữ liệu từ file u.genre vào DataFrame
genre_data = spark.read.format("csv").option("delimiter", "|").load("dbfs:/FileStore/shared_uploads/danghia2002@gmail.com/u.genre")
genre_data = genre_data.selectExpr("_c0 as movie_classification", "_c1 as id_classification")
genre_data = genre_data.withColumn("id_classification", genre_data["id_classification"].cast("integer"))

In [ ]:
genre_data.show()


+--------------------+-----------------+
|movie_classification|id_classification|
+--------------------+-----------------+
|             unknown|                0|
|              Action|                1|
|           Adventure|                2|
|           Animation|                3|
|          Children's|                4|
|              Comedy|                5|
|               Crime|                6|
|         Documentary|                7|
|               Drama|                8|
|             Fantasy|                9|
|           Film-Noir|               10|
|              Horror|               11|
|             Musical|               12|
|             Mystery|               13|
|             Romance|               14|
|              Sci-Fi|               15|
|            Thriller|               16|
|                 War|               17|
|             Western|               18|
+--------------------+-----------------+



In [ ]:
#Thông tin thống kê về rating của người dùng về các bộ phim
rating_data.describe("rating").show()

+-------+------------------+
|summary|            rating|
+-------+------------------+
|  count|            100000|
|   mean|           3.52986|
| stddev|1.1256735991443214|
|    min|                 1|
|    max|                 5|
+-------+------------------+



In [ ]:
#Người dùng 1 đã rating những bộ phim nào
user1_ratings = rating_data.filter(rating_data["user_id"] == 1)
user1_ratings.show()

+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|      1|     61|     4|878542420|
|      1|    189|     3|888732928|
|      1|     33|     4|878542699|
|      1|    160|     4|875072547|
|      1|     20|     4|887431883|
|      1|    202|     5|875072442|
|      1|    171|     5|889751711|
|      1|    265|     4|878542441|
|      1|    155|     2|878542201|
|      1|    117|     3|874965739|
|      1|     47|     4|875072125|
|      1|    222|     4|878873388|
|      1|    253|     5|874965970|
|      1|    113|     5|878542738|
|      1|    227|     4|876892946|
|      1|     17|     3|875073198|
|      1|     90|     4|878542300|
|      1|     64|     5|875072404|
|      1|     92|     3|876892425|
|      1|    228|     5|878543541|
+-------+-------+------+---------+
only showing top 20 rows



In [ ]:
#Tên bộ phim có mã 1
item1_title = item_data.filter(item_data["item_id"] == 1).select("title").first()[0]
print("Tên bộ phim có mã 1:", item1_title)

Tên bộ phim có mã 1: Toy Story (1995)


Số người rating: 182
Rating cao nhất: 5
Rating thấp nhất: 2
Rating trung bình: 4.1208791208791204


In [ ]:
#Số lượt, trung bình, cao nhất, thấp nhất rating từng bộ phim
rating_counts = rating_data.groupBy("item_id").count()
rating_stats = rating_data.groupBy("item_id").agg({"rating": "avg", "rating": "max", "rating": "min"})
rating_data_stats = item_data.join(rating_counts, "item_id").join(rating_stats, "item_id")
rating_data_stats.show()

+-------+--------------------+------------+--------------------+-----+-----------+
|item_id|               title|release_date|            imdb_url|count|min(rating)|
+-------+--------------------+------------+--------------------+-----+-----------+
|    496|It's a Wonderful ...| 01-Jan-1946|http://us.imdb.co...|  231|          1|
|    471|Courage Under Fir...| 08-Mar-1996|http://us.imdb.co...|  221|          1|
|    463|Secret of Roan In...| 01-Jan-1994|http://us.imdb.co...|   71|          1|
|    148|Ghost and the Dar...| 11-Oct-1996|http://us.imdb.co...|  128|          1|
|   1342|Convent, The (Con...| 14-Jun-1996|http://us.imdb.co...|    2|          1|
|    833|  Bulletproof (1996)| 06-Sep-1996|http://us.imdb.co...|   49|          1|
|   1088|  Double Team (1997)| 04-Apr-1997|http://us.imdb.co...|   13|          1|
|   1591|Duoluo tianshi (1...| 21-Jan-1998|http://us.imdb.co...|    6|          1|
|   1238|   Full Speed (1996)| 01-Jan-1996|http://us.imdb.co...|    8|          2|
|   

In [ ]:
#Top 10 bộ phim có số lượt rating cao nhất
top_10_movies = rating_counts.orderBy("count", ascending=False).limit(10)
top_10_movies_data = top_10_movies.join(item_data, "item_id")
top_10_movies_data.show()

+-------+-----+--------------------+------------+--------------------+
|item_id|count|               title|release_date|            imdb_url|
+-------+-----+--------------------+------------+--------------------+
|      1|  452|    Toy Story (1995)| 01-Jan-1995|http://us.imdb.co...|
|     50|  583|    Star Wars (1977)| 01-Jan-1977|http://us.imdb.co...|
|    100|  508|        Fargo (1996)| 14-Feb-1997|http://us.imdb.co...|
|    121|  429|Independence Day ...| 03-Jul-1996|http://us.imdb.co...|
|    181|  507|Return of the Jed...| 14-Mar-1997|http://us.imdb.co...|
|    258|  509|      Contact (1997)| 11-Jul-1997|http://us.imdb.co...|
|    286|  481|English Patient, ...| 15-Nov-1996|http://us.imdb.co...|
|    288|  478|       Scream (1996)| 20-Dec-1996|http://us.imdb.co...|
|    294|  485|    Liar Liar (1997)| 21-Mar-1997|http://us.imdb.co...|
|    300|  431|Air Force One (1997)| 01-Jan-1997|http://us.imdb.co...|
+-------+-----+--------------------+------------+--------------------+



In [ ]:
#Các bộ phim có ít hơn 10 lượt rating
less_than_10_movies = rating_counts.filter(rating_counts["count"] < 10)
less_than_10_movies_data = less_than_10_movies.join(item_data, "item_id")
less_than_10_movies_data.show()


+-------+-----+--------------------+------------+--------------------+
|item_id|count|               title|release_date|            imdb_url|
+-------+-----+--------------------+------------+--------------------+
|   1342|    2|Convent, The (Con...| 14-Jun-1996|http://us.imdb.co...|
|   1591|    6|Duoluo tianshi (1...| 21-Jan-1998|http://us.imdb.co...|
|   1238|    8|   Full Speed (1996)| 01-Jan-1996|http://us.imdb.co...|
|   1580|    1|     Liebelei (1933)| 01-Jan-1933|http://us.imdb.co...|
|   1645|    1|Butcher Boy, The ...| 01-Jan-1998|http://us.imdb.co...|
|    858|    3|Amityville: Dollh...| 01-Jan-1996|http://us.imdb.co...|
|   1395|    6|Hurricane Streets...| 01-Jan-1998|http://us.imdb.co...|
|   1522|    7|Trial by Jury (1994)| 01-Jan-1994|http://us.imdb.co...|
|   1507|    1|Three Lives and O...| 11-Oct-1996|http://us.imdb.co...|
|    897|    2| Time Tracers (1995)| 01-Jan-1995|http://us.imdb.co...|
|   1460|    1|    Sleepover (1995)| 25-Oct-1996|http://us.imdb.co...|
|   13

In [ ]:
# Tên các bộ phim có số lượt rating cao nhất
top_movies_titles = top_10_movies_data.select("title")
top_movies_titles.show(truncate=False)


+-----------------------------+
|title                        |
+-----------------------------+
|Toy Story (1995)             |
|Star Wars (1977)             |
|Fargo (1996)                 |
|Independence Day (ID4) (1996)|
|Return of the Jedi (1983)    |
|Contact (1997)               |
|English Patient, The (1996)  |
|Scream (1996)                |
|Liar Liar (1997)             |
|Air Force One (1997)         |
+-----------------------------+



In [ ]:
#Những người dùng nghề 'programmer' rating >= 4 những thể loại phim nào
programmer_ratings = rating_data.join(user_data, ["user_id"], "inner").filter(
(user_data["occupation"] == "programmer") & (rating_data["rating"] >= 4)
).join(item_data, ["item_id"], "inner").join(genre_data, ["id_classification"], "inner").groupBy("_classification").count()
programmer_ratings.show()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-852978552748527>:1
----> 1 programmer_ratings = rating_data.join(user_data, ["user_id"], "inner").filter(
      2     (user_data["occupation"] == "programmer") & (rating_data["rating"] >= 4)
      3 ).join(item_data, ["item_id"], "inner").join(genre_data, ["id_classification"], "inner").groupBy("_classification").count()
      4 programmer_ratings.show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:2364, in DataFrame.join(self, other, on, how)
  

In [ ]:
#Những bộ phim mà người dùng 1 chưa rating
user1_rated_movies = rating_data.filter(rating_data["user_id"] == 1).select("item_id")
unrated_movies = item_data.join(user1_rated_movies, ["item_id"], "leftanti")
unrated_movies.show()

+-------+--------------------+------------+--------------------+
|item_id|               title|release_date|            imdb_url|
+-------+--------------------+------------+--------------------+
|    273|         Heat (1995)| 01-Jan-1995|http://us.imdb.co...|
|    274|      Sabrina (1995)| 01-Jan-1995|http://us.imdb.co...|
|    275|Sense and Sensibi...| 01-Jan-1995|http://us.imdb.co...|
|    276|Leaving Las Vegas...| 01-Jan-1995|http://us.imdb.co...|
|    277|  Restoration (1995)| 01-Jan-1995|http://us.imdb.co...|
|    278| Bed of Roses (1996)| 01-Jan-1996|http://us.imdb.co...|
|    279|Once Upon a Time....| 01-Jan-1995|http://us.imdb.co...|
|    280|Up Close and Pers...| 01-Mar-1996|http://us.imdb.co...|
|    281|River Wild, The (...| 01-Jan-1994|http://us.imdb.co...|
|    282|Time to Kill, A (...| 13-Jul-1996|http://us.imdb.co...|
|    283|         Emma (1996)| 02-Aug-1996|http://us.imdb.co...|
|    284|      Tin Cup (1996)| 16-Aug-1996|http://us.imdb.co...|
|    285|Secrets & Lies (

In [ ]:
#Những bộ phim mà tất cả rating >= 4
from pyspark.sql import functions as F
highly_rated_movies = rating_data.filter(rating_data["rating"] >= 4).groupBy("item_id").agg(
    F.count("rating").alias("total_ratings")
).join(item_data, ["item_id"], "inner").filter(
    F.col("total_ratings") == rating_data.select("user_id").distinct().count()
).select("title")
highly_rated_movies.show()

+-----+
|title|
+-----+
+-----+



In [ ]:
#Những bộ phim có độ lệch rating giữa các người dùng không quá 2
rating_deviation = rating_data.groupBy("item_id").agg(
    F.stddev("rating").alias("rating_deviation")
).filter(F.col("rating_deviation") <= 2).join(item_data, ["item_id"], "inner").select("title")
rating_deviation.show()

+--------------------+
|               title|
+--------------------+
|It's a Wonderful ...|
|Courage Under Fir...|
|Secret of Roan In...|
|Ghost and the Dar...|
|  Bulletproof (1996)|
|  Double Team (1997)|
|Duoluo tianshi (1...|
|   Full Speed (1996)|
|Man Without a Fac...|
|Angels in the Out...|
|  Money Train (1995)|
|Amityville: Dollh...|
|       Sirens (1994)|
|Jungle2Jungle (1997)|
|Fire Down Below (...|
|Anne Frank Rememb...|
|Truman Show, The ...|
|Hurricane Streets...|
|Trial by Jury (1994)|
|Man in the Iron M...|
+--------------------+
only showing top 20 rows



In [ ]:

#Những bộ phim người dùng 1 đã rating nhưng người dùng 2 không rating
user1_ratings = rating_data.filter(rating_data["user_id"] == 1).select("item_id")
user2_unrated_movies = rating_data.filter(rating_data["user_id"] == 2).select("item_id").join(
    user1_ratings, ["item_id"], "leftanti"
).join(item_data, ["item_id"], "inner").select("title")
user2_unrated_movies.show()

+--------------------+
|               title|
+--------------------+
|     Rosewood (1997)|
|3 Ninjas: High No...|
|  Ulee's Gold (1997)|
|Fierce Creatures ...|
|Midnight in the G...|
|River Wild, The (...|
|Up Close and Pers...|
|  Ulee's Gold (1997)|
|FairyTale: A True...|
|Devil's Advocate,...|
|As Good As It Get...|
|    Apt Pupil (1998)|
|     In & Out (1997)|
|      Titanic (1997)|
|Once Upon a Time....|
|      Hoodlum (1997)|
|     Face/Off (1997)|
|  Restoration (1995)|
|Time to Kill, A (...|
|    Breakdown (1997)|
+--------------------+
only showing top 20 rows



In [ ]:
#Những bộ phim rating chung của người dùng 1 và 2
user1_ratings = rating_data.filter(rating_data["user_id"] == 1).select("item_id")
user2_ratings = rating_data.filter(rating_data["user_id"] == 2).select("item_id")
common_movies = user1_ratings.join(user2_ratings, ["item_id"], "inner").join(
    item_data, ["item_id"], "inner"
).select("title")
common_movies.show()

+--------------------+
|               title|
+--------------------+
|Good Will Hunting...|
|      Contact (1997)|
|  Postino, Il (1994)|
|        Kolya (1996)|
|  Richard III (1995)|
|My Best Friend's ...|
|        Fargo (1996)|
|Full Monty, The (...|
|    Toy Story (1995)|
| Men in Black (1997)|
|Jerry Maguire (1996)|
|    Star Wars (1977)|
|Godfather, The (1...|
|Birdcage, The (1996)|
|Shall We Dance? (...|
|Antonia's Line (1...|
|Truth About Cats ...|
|Mighty Aphrodite ...|
+--------------------+



In [ ]:
#Những bộ phim rating chung của từng cặp người dùng
user_ratings = rating_data.groupBy("item_id").agg(
    F.collect_set("user_id").alias("users")
).filter(F.size("users") > 1).join(
    item_data, ["item_id"], "inner"
).select("title", "users")
user_ratings.show()

+--------------------+--------------------+
|               title|               users|
+--------------------+--------------------+
|    Toy Story (1995)|[843, 793, 785, 7...|
|    GoldenEye (1995)|[466, 102, 387, 7...|
|   Four Rooms (1995)|[81, 793, 916, 10...|
|   Get Shorty (1995)|[466, 233, 102, 3...|
|      Copycat (1995)|[102, 437, 814, 9...|
|Shanghai Triad (Y...|[9, 655, 524, 1, ...|
|Twelve Monkeys (1...|[843, 466, 814, 7...|
|         Babe (1995)|[233, 610, 437, 3...|
|Dead Man Walking ...|[793, 764, 277, 2...|
|  Richard III (1995)|[662, 387, 2, 104...|
|Seven (Se7en) (1995)|[466, 437, 764, 3...|
|Usual Suspects, T...|[437, 785, 387, 2...|
|Mighty Aphrodite ...|[102, 437, 560, 3...|
|  Postino, Il (1994)|[233, 437, 306, 7...|
|Mr. Holland's Opu...|[437, 764, 277, 2...|
|French Twist (Gaz...|[305, 894, 763, 2...|
|From Dusk Till Da...|[466, 814, 916, 3...|
|White Balloon, Th...|[15, 655, 1, 90, ...|
|Antonia's Line (1...|[539, 52, 306, 2,...|
|Angels and Insect...|[458, 387,